## 3.6　ニューラルネットワークの構造：基本的な Keras API

In [1]:
import math
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.datasets import mnist

#### 3.6.1　層：ディープラーニングに欠かせない要素

In [2]:
class SimpleDense(keras.layers.Layer):
  def __init__(self, units, activation=None):
    super().__init__()
    self.units = units
    self.activation = activation

  def build(self, input_shape):
    input_dim = input_shape[-1]
    self.W = self.add_weight(shape=(input_dim, self.units), initializer='random_normal')
    self.b = self.add_weight(shape=(self.units,), initializer='zeros')

  def call(self, inputs):
    y = tf.matmul(inputs, self.W) + self.b
    if self.activation is not None:
      y = self.activation(y)
    return y

In [3]:
my_dense = SimpleDense(units=32, activation=tf.nn.relu)
input_tensor = tf.ones(shape=(2, 784))
output_tensor = my_dense(input_tensor)
print(output_tensor.shape)

(2, 32)


In [4]:
model = keras.Sequential([
    SimpleDense(32, activation='relu'),
    SimpleDense(64, activation='relu'),
    SimpleDense(32, activation='relu'),
    SimpleDense(10, activation='softmax')
])

#### 3.6.3　compile メソッド：学習プロセスを設定する

In [5]:
model = keras.Sequential([keras.layers.Dense(1)])
model.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['accuracy'])
model.compile(optimizer=keras.optimizers.RMSprop(), loss=keras.losses.MeanSquaredError(), metrics=[keras.metrics.BinaryAccuracy()])

#### 3.6.5　fit メソッド：モデルを適合させる

In [6]:
num_samples_per_class = 1000
negative_samples = np.random.multivariate_normal(mean=[0, 3], cov=[[1, 0.5], [0.5, 1]], size=num_samples_per_class)
positive_samples = np.random.multivariate_normal(mean=[3, 0], cov=[[1, 0.5], [0.5, 1]], size=num_samples_per_class)

In [7]:
inputs = np.vstack((negative_samples, positive_samples)).astype(np.float32)
targets = np.vstack((np.zeros((num_samples_per_class, 1), dtype='float32'), np.ones((num_samples_per_class, 1), dtype='float32')))
print(inputs.shape)
print(targets.shape)

(2000, 2)
(2000, 1)


In [8]:
history = model.fit(inputs, targets, epochs=5, batch_size=128)

Epoch 1/5
16/16 [==============================] - 1s 2ms/step - loss: 5.9414 - binary_accuracy: 0.6935
Epoch 2/5
16/16 [==============================] - 0s 2ms/step - loss: 5.5996 - binary_accuracy: 0.7050
Epoch 3/5
16/16 [==============================] - 0s 2ms/step - loss: 5.3180 - binary_accuracy: 0.7140
Epoch 4/5
16/16 [==============================] - 0s 3ms/step - loss: 5.0534 - binary_accuracy: 0.7240
Epoch 5/5
16/16 [==============================] - 0s 5ms/step - loss: 4.7971 - binary_accuracy: 0.7310


In [9]:
history.history

{'loss': [5.941409587860107,
  5.599635124206543,
  5.317971229553223,
  5.053441524505615,
  4.797116756439209],
 'binary_accuracy': [0.6934999823570251,
  0.7049999833106995,
  0.7139999866485596,
  0.7239999771118164,
  0.7310000061988831]}

#### 3.6.6　検証データでの損失値と指標を監視する

In [10]:
model = keras.Sequential([keras.layers.Dense(1)])
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.1), loss=keras.losses.MeanSquaredError(), metrics=[keras.metrics.BinaryAccuracy()])

In [11]:
indices_permutation = np.random.permutation(len(inputs))
shuffled_inputs = inputs[indices_permutation]
shuffled_targets = targets[indices_permutation]

In [12]:
num_validation_samples = int(0.3 * len(inputs))
val_inputs = shuffled_inputs[:num_validation_samples]
val_targets = shuffled_targets[:num_validation_samples]
training_inputs = shuffled_inputs[num_validation_samples:]
training_targets = shuffled_targets[num_validation_samples:]

In [13]:
model.fit(training_inputs, training_targets, epochs=5, batch_size=16, validation_data=(val_inputs, val_targets))

Epoch 1/5
88/88 [==============================] - 1s 7ms/step - loss: 0.1264 - binary_accuracy: 0.9464 - val_loss: 0.1276 - val_binary_accuracy: 0.9233
Epoch 2/5
88/88 [==============================] - 0s 3ms/step - loss: 0.0696 - binary_accuracy: 0.9621 - val_loss: 0.0545 - val_binary_accuracy: 0.9900
Epoch 3/5
88/88 [==============================] - 0s 3ms/step - loss: 0.0723 - binary_accuracy: 0.9579 - val_loss: 0.1259 - val_binary_accuracy: 0.8450
Epoch 4/5
88/88 [==============================] - 0s 3ms/step - loss: 0.0686 - binary_accuracy: 0.9657 - val_loss: 0.0298 - val_binary_accuracy: 1.0000
Epoch 5/5
88/88 [==============================] - 0s 3ms/step - loss: 0.0694 - binary_accuracy: 0.9536 - val_loss: 0.1735 - val_binary_accuracy: 0.9200


In [14]:
loss_and_metrics = model.evaluate(val_inputs, val_targets, batch_size=128)

5/5 [==============================] - 0s 4ms/step - loss: 0.1735 - binary_accuracy: 0.9200


#### 3.6.7　predict メソッド：訓練したモデルを使う

In [15]:
predictions = model.predict(val_inputs, batch_size=128)
print(predictions[:10])

5/5 [==============================] - 0s 4ms/step
[[1.3773763 ]
 [0.3511408 ]
 [1.1348267 ]
 [1.0732017 ]
 [0.5578917 ]
 [0.8886408 ]
 [1.5624809 ]
 [1.4129629 ]
 [0.03852242]
 [0.33096266]]
